# Análise do LINPACK Benchmark


## 1 - Lendo o arquivo e coletando os dados

 - O arquivo é estruturado com 12 testes de cada combinação de parâmetros (N, NB). Assim, usaremos uma string com ambas as informações como chave de uma lista de tuplas _(tempo-de-parede,gflops)_. Ex: 1000_32 -> [(0.03,2.9828)...]

 - São 45 sets de parâmetros, com 12 rodadas cada uma. 

In [35]:
import re
from functools import reduce
MAX_LINE_TO_READ = 5445


data = {}

In [36]:
with open("HPL.out", 'r') as f:

    counter = 45

    for i in range(44): # linhas de introdução para pular
        f.readline()

    for i in range(45): # 540/12 = 45 sets de teste
        
        for j in range(12):

            if(counter == MAX_LINE_TO_READ):
                break

            for k in range(3):
                f.readline() # lê o cabeçalho
                counter += 1

            # PARTE IMPORTANTE!!!
            info_line = f.readline()
            counter += 1 
            
            info = re.split(r'\s+', info_line)
            _,n, nb, _, _, time, gflops,_ = info
            
            time = float(time.strip())

            base,expo = gflops.split("e+") 
            base,expo = float(base),int(expo[1:])

            gflops = pow(base,expo)

            label=f"{n}_{nb}"
            
            if label in data:
                data[label].append((time,gflops))

            else:
                data[label] = [(time,gflops)]

            # lê o restante das informações
            for m in range(6):
                f.readline()
                counter += 1


In [37]:
data["10000_128"]

[(10.36, 6.4362),
 (10.19, 6.5431),
 (10.17, 6.5547),
 (10.23, 6.5152),
 (10.26, 6.4982),
 (10.4, 6.4137),
 (10.33, 6.4554),
 (10.3, 6.4737),
 (10.61, 6.2864),
 (10.26, 6.4972),
 (10.34, 6.4471),
 (10.33, 6.452)]

## 2 - Obtendo as médias (tempo e GFLOPS)

In [38]:
for key, values in data.items():

    time_sum = reduce(lambda t_acc,x: t_acc + x[0],values,0)
    gflop_sum = reduce(lambda gfl_acc,x: gfl_acc + x[1],values,0)

    mean_time = time_sum/len(values)
    mean_gflop = gflop_sum/len(values)

    data[key] = (mean_time, mean_gflop)


In [39]:
data["10000_128"]

(10.315, 6.464408333333334)